In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.2 MB/s eta 0:00:00


In [ ]:
!pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.5 MB/s eta 0:00:00


In [ ]:
#import the reqired libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from transformers import BertModel, BertTokenizer
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from sklearn.model_selection import train_test_split
import torch
import re
import math
import plotly.express as px
from tqdm import tqdm
from sklearn.feature_selection import RFECV
from sklearn import metrics
from scipy.stats import spearmanr
from sklearn.metrics import r2_score
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression  
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
import catboost as cb
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
#read the files
train_df=pd.read_csv('train.csv')
train_updates_df=pd.read_csv('train_updates_20220929.csv')
test_df=pd.read_csv("test.csv")


## Exploratory Data Anaysis


In [ ]:
train_df.head()

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


In [ ]:
train_df.shape

(31390, 5)

In [ ]:
train_updates_df.head()

,seq_id,protein_sequence,pH,data_source,tm
0,69,NaN,NaN,NaN,NaN
1,70,NaN,NaN,NaN,NaN
2,71,NaN,NaN,NaN,NaN
3,72,NaN,NaN,NaN,NaN
4,73,NaN,NaN,NaN,NaN


In [ ]:
train_updates_df.shape

(2434, 5)

In [ ]:
for i in train_updates_df['seq_id'] :
    train_df[train_df['seq_id'] == i] = train_updates_df[train_updates_df['seq_id'] == i]

In [ ]:
train_df.shape

(31390, 5)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            28956 non-null  float64
 1   protein_sequence  28956 non-null  object 
 2   pH                28670 non-null  float64
 3   data_source       28001 non-null  object 
 4   tm                28956 non-null  float64
dtypes: float64(3), object(2)
memory usage: 1.2+ MB


In [ ]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
seq_id,28956.0,15744.916529,9251.179714,0.00,7526.75,15587.5,23902.25,31389.0
pH,28670.0,6.872918,0.792550,1.99,7.00,7.0,7.00,11.0
tm,28956.0,51.360399,12.060858,25.10,43.60,48.8,54.60,130.0


In [ ]:
#check for null values
train_df.isnull().sum()

seq_id              2434
protein_sequence    2434
pH                  2720
data_source         3389
tm                  2434
dtype: int64

In [ ]:
train_df=train_df.dropna(how='all')

In [ ]:
train_df.isnull().sum()

seq_id                0
protein_sequence      0
pH                  286
data_source         955
tm                    0
dtype: int64

In [ ]:
train_df['pH'] = train_df['pH'].fillna(train_df['pH'].mean())

<ipython-input-23-fa1860bb7f95>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['pH'] = train_df['pH'].fillna(train_df['pH'].mean())


In [ ]:
train_df.shape

(28956, 5)

In [ ]:
px.histogram(train_df, x='tm', title="tm Distribution", template='plotly_dark',width=800,height=400)

In [ ]:
px.histogram(train_df, x='pH', title='pH Count', template='presentation')

In [ ]:
train_df['protein_sequence'][0]

'AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVGMIKDAGDDPDVTHGAEIQAFVRFASEDRLEGGEGVGVVTKPGLGVPVGEPAINPVPRRMIWEAVREVTERPLAVTIAIPGGEELAKKTLNPRLGILGGLSVLGTTGVVKPYSTSAFRMSVVQAVGVARANGLLEIAATTGGKSERFAQRLLPHLPEMAFIEMGDFVGDVLRAARKVGVEVVRVVGMIGKISKMADGKTMTHAAGGEVNLSLLLSLLKEAGASPKALKEAEGAATARRFLEIALEEGLELFFVNLVRLAQEKLQAYIGERPFVSVALTDFDEGRCLAAWPDREVYR'

In [ ]:
#check and remove duplicates
train_df.duplicated(subset=['protein_sequence','pH','data_source']).sum()

350

In [ ]:
train_df.drop_duplicates(subset=['protein_sequence','pH','data_source'],inplace=True)

In [ ]:
train_df = train_df.drop(['data_source'],axis=1)

In [ ]:
train_df.shape

(28606, 4)

In [ ]:
train_df.head(3)

,seq_id,protein_sequence,pH,tm
0,0.0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,1.0,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,2.0,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5


In [ ]:
amino_count = train_df['protein_sequence'].str.split('').explode('protein_sequence').value_counts().drop('')
amino_count

L    1231815
A     991500
E     938883
S     936806
G     865000
V     841697
K     804514
D     721148
R     696066
I     692793
T     686305
P     651993
Q     566942
N     547159
F     487035
Y     380580
M     302394
H     296074
C     190103
W     144002
Name: protein_sequence, dtype: int64

In [ ]:
fig = px.bar(amino_count, x=amino_count.index, y='protein_sequence', color=amino_count.index)
fig.update_layout(
    title='Amino Acid Count',
    height=600,
    template='ggplot2'
)
fig.show()

## Hand Engineering


In [ ]:
# Protein Sequence Length as a column
train_df["protein_length"] = train_df["protein_sequence"].apply(lambda x: len(x))

In [ ]:
train_df.head(3)

,seq_id,protein_sequence,pH,tm,protein_length
0,0.0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,341
1,1.0,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,286
2,2.0,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,497


In [ ]:
def return_amino_acid_df(df):
  # Feature Engineering on Train Data
  amino_acids=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
  for amino_acid in amino_acids:
    df[amino_acid]=df['protein_sequence'].str.count(amino_acid,re.I)/df['protein_length']
    #df[amino_acid]=df['protein_sequence'].str.count(amino_acid,re.I)
  return df

In [ ]:
train_df = return_amino_acid_df(train_df)

In [ ]:
train_df.head()

,seq_id,protein_sequence,pH,tm,protein_length,A,C,D,E,F,...,M,N,P,Q,R,S,T,V,W,Y
0,0.0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,341,0.131965,0.002933,0.038123,0.087977,0.038123,...,0.023460,0.014663,0.052786,0.017595,0.073314,0.032258,0.041056,0.108504,0.011730,0.008798
1,1.0,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,286,0.097902,0.000000,0.034965,0.181818,0.020979,...,0.006993,0.020979,0.027972,0.076923,0.104895,0.048951,0.041958,0.045455,0.010490,0.010490
2,2.0,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,497,0.100604,0.018109,0.054326,0.064386,0.042254,...,0.012072,0.030181,0.040241,0.050302,0.062374,0.066398,0.060362,0.060362,0.006036,0.032193
3,3.0,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,265,0.075472,0.018868,0.071698,0.109434,0.045283,...,0.007547,0.033962,0.060377,0.033962,0.037736,0.060377,0.071698,0.052830,0.011321,0.015094
4,4.0,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,1451,0.059269,0.009649,0.053756,0.053756,0.022054,...,0.021365,0.044797,0.088215,0.037216,0.043418,0.101999,0.082702,0.085458,0.011027,0.032391


In [ ]:
# PhysioChemical Properties of Amino acids

#Aromaticity
def calculate_aromaticity(row):
  sequence = str(row[1])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.aromaticity()

#Molecular Weight
def calculate_molecular_weight(row):
  sequence = str(row[1])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.molecular_weight()

#Instability Index
def calculate_instability_index(row):
  sequence = str(row[1])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.instability_index()

#Hydrophobicity
def calculate_hydrophobicity(row):
  sequence = str(row[1])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.gravy(scale='KyteDoolitle')

#Isoelectric Point
def calculate_isoelectric_point(row):
  sequence = str(row[1])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.isoelectric_point()

#Charge
def calculate_charge(row):
  sequence = str(row[1])
  X = ProteinAnalysis(sequence)
  return "%0.2f" % X.charge_at_pH(row[2])

In [ ]:
train_df['Aromaticity'] = train_df.apply(calculate_aromaticity, axis=1)
train_df['Molecular Weight'] = train_df.apply(calculate_molecular_weight, axis=1)
train_df['Instability Index'] = train_df.apply(calculate_instability_index, axis=1)
train_df['Hydrophobicity'] = train_df.apply(calculate_hydrophobicity, axis=1)
train_df['Isoelectric Point'] = train_df.apply(calculate_isoelectric_point, axis=1)
train_df['Charge'] = train_df.apply(calculate_charge, axis=1)


In [ ]:
train_df.head()

,seq_id,protein_sequence,pH,tm,protein_length,A,C,D,E,F,...,T,V,W,Y,Aromaticity,Molecular Weight,Instability Index,Hydrophobicity,Isoelectric Point,Charge
0,0.0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,341,0.131965,0.002933,0.038123,0.087977,0.038123,...,0.041056,0.108504,0.011730,0.008798,0.06,36320.72,28.39,0.15,6.11,-1.87
1,1.0,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,286,0.097902,0.000000,0.034965,0.181818,0.020979,...,0.041958,0.045455,0.010490,0.010490,0.04,32837.99,65.11,-1.09,5.14,-12.72
2,2.0,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,497,0.100604,0.018109,0.054326,0.064386,0.042254,...,0.060362,0.060362,0.006036,0.032193,0.08,53428.80,35.09,-0.71,9.03,11.73
3,3.0,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,265,0.075472,0.018868,0.071698,0.109434,0.045283,...,0.071698,0.052830,0.011321,0.015094,0.07,29475.60,50.50,-0.51,4.68,-20.56
4,4.0,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,1451,0.059269,0.009649,0.053756,0.053756,0.022054,...,0.082702,0.085458,0.011027,0.032391,0.07,158761.98,45.67,-0.40,5.92,-21.66


In [ ]:
train_df.drop(columns=["protein_length"], inplace=True)
train_df.drop(columns=["protein_sequence", "seq_id"], inplace=True)

In [ ]:
# Reset the DataFrame indexes
train_df.reset_index(drop=True, inplace=True)

In [ ]:
train_df['Aromaticity'] = pd.to_numeric(train_df['Aromaticity'])
train_df['Molecular Weight'] = pd.to_numeric(train_df['Molecular Weight'])
train_df['Instability Index'] = pd.to_numeric(train_df['Instability Index'])
train_df['Hydrophobicity'] = pd.to_numeric(train_df['Hydrophobicity'])
train_df['Isoelectric Point'] = pd.to_numeric(train_df['Isoelectric Point'])
train_df['Charge'] = pd.to_numeric(train_df['Charge'])

In [ ]:
train_df.head(3)

,pH,tm,A,C,D,E,F,G,H,I,...,T,V,W,Y,Aromaticity,Molecular Weight,Instability Index,Hydrophobicity,Isoelectric Point,Charge
0,7.0,75.7,0.131965,0.002933,0.038123,0.087977,0.038123,0.111437,0.008798,0.041056,...,0.041056,0.108504,0.011730,0.008798,0.06,36320.72,28.39,0.15,6.11,-1.87
1,7.0,50.5,0.097902,0.000000,0.034965,0.181818,0.020979,0.062937,0.013986,0.045455,...,0.041958,0.045455,0.010490,0.010490,0.04,32837.99,65.11,-1.09,5.14,-12.72
2,7.0,40.5,0.100604,0.018109,0.054326,0.064386,0.042254,0.130785,0.022133,0.032193,...,0.060362,0.060362,0.006036,0.032193,0.08,53428.80,35.09,-0.71,9.03,11.73


## Model 1

In [ ]:
# Split the data into train and validation set
train_df_new, rem_df = train_test_split(train_df, train_size=0.8,random_state=99)

val_df, test_df = train_test_split(rem_df, test_size=0.5,random_state=123)

In [ ]:
X_train = train_df.drop(columns=['tm'])
y_train = train_df['tm']

X_val = val_df.drop(columns=['tm'])
y_val = val_df['tm']

X_test = test_df.drop(columns=['tm'])
y_test = test_df['tm']


In [ ]:
#XGBoost Model
model = XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=200, tree_method="gpu_hist",random_state=98)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=98, ...)

In [ ]:
rfecv = RFECV(estimator= model, step = 5, cv = 5, scoring='neg_mean_squared_error')
rfecv = rfecv.fit(X_train, y_train)

print("The optimal number of features:", rfecv.n_features_)
print('Selected features: %s' % list(X_train.columns[rfecv.support_]))

best_features = list(X_train.columns[rfecv.support_])

In [ ]:
X_train_new = X_train[best_features]
X_val_new = X_val[best_features]
X_test_new = X_test[best_features]

In [ ]:
model1 = XGBRegressor(learning_rate=0.1, max_depth=20, n_estimators=250, tree_method="gpu_hist",random_state=123)
model1.fit(X_train_new, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=20, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=250, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=123, ...)

In [ ]:
y_pred_train = model1.predict(X_train_new)
y_pred_val = model1.predict(X_val_new)

mse_train=mean_squared_error(y_train, y_pred_train)
mse_val=mean_squared_error(y_val, y_pred_val)

print("Root Mean Squared error on Train data is:",math.sqrt(mse_train))
print("Root Mean Squared error on Val data is:",math.sqrt(mse_val))

Root Mean Squared error on Train data is: 1.0510422638776984
Root Mean Squared error on Val data is: 1.045781970944786


In [ ]:
# Results with the best model
pred1 = model1.predict(X_test_new)
mse_test=mean_squared_error(y_test, pred1)
print("Root Mean Squared error on Test data is:",math.sqrt(mse_test))

Root Mean Squared error on Test data is: 0.8573355940014015


In [ ]:
spearmanr(y_test, pred1)

SignificanceResult(statistic=0.9969548930747925, pvalue=0.0)

In [ ]:
eval_df = pd.DataFrame({"Actual": y_test.values, "Predicted": pred1})
spearmanr_val = spearmanr(eval_df["Actual"], eval_df["Predicted"])

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x=eval_df["Actual"],
        y=eval_df["Predicted"],
        name='Actual vs. Predicted',
        mode='markers'
    )
)

fig.add_annotation(
    x=0.95,
    y=0.10,
    xref='paper',
    yref='paper',
    text='Spearman Rank Correlation = {:.3f}'.format(spearmanr_val.correlation),
    showarrow=False
)

fig.update_layout(
    title='Actual vs. Predicted',
    height=700
)

fig.show()

In [ ]:
r2_train = r2_score(y_train, y_pred_train)
r2_val = r2_score(y_val, y_pred_val)
r2_test = r2_score(y_test, pred1)
print('r2 score for train is', r2_train)
print('r2 score for val is', r2_val)
print('r2 score for test is', r2_test)

r2 score for train is 0.9924098344859508
r2 score for val is 0.9922527783763758
r2 score for test is 0.994946838327278


In [ ]:
print('Mean Absolute Error Train:', metrics.mean_absolute_error(y_train, y_pred_train))
print('Mean Absolute Error Val:', metrics.mean_absolute_error(y_val, y_pred_val))
print('Mean Absolute Error Test:', metrics.mean_absolute_error(y_test, pred1))

Mean Absolute Error Train: 0.1436972944217904
Mean Absolute Error Val: 0.14239589087157264
Mean Absolute Error Test: 0.11034466537800916


## Model 2

In [ ]:
# Split the data into train and validation set
train_df_new, rem_df = train_test_split(train_df, train_size=0.8,random_state=98)

val_df, test_df = train_test_split(rem_df, test_size=0.5,random_state=123)

In [ ]:
X_train = train_df.drop(columns=['tm'])
y_train = train_df['tm']

X_val = val_df.drop(columns=['tm'])
y_val = val_df['tm']

X_test = test_df.drop(columns=['tm'])
y_test = test_df['tm']


In [ ]:
#XGBoost Model
model = XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=200, tree_method="gpu_hist",random_state=98)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=98, ...)

In [ ]:
rfecv = RFECV(estimator= model, step = 5, cv = 5, scoring='neg_mean_squared_error')
rfecv = rfecv.fit(X_train, y_train)

print("The optimal number of features:", rfecv.n_features_)
print('Selected features: %s' % list(X_train.columns[rfecv.support_]))

best_features = list(X_train.columns[rfecv.support_])

The optimal number of features: 27
Selected features: ['pH', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'Aromaticity', 'Molecular Weight', 'Instability Index', 'Hydrophobicity', 'Isoelectric Point', 'Charge']


In [ ]:
X_train_new = X_train[best_features]
X_val_new = X_val[best_features]
X_test_new = X_test[best_features]

In [ ]:
model2 = XGBRegressor(learning_rate=0.1, max_depth=20, n_estimators=250, tree_method="gpu_hist",random_state=123)
model2.fit(X_train_new, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=20, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=250, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=123, ...)

In [ ]:
y_pred_train = model2.predict(X_train_new)
y_pred_val = model2.predict(X_val_new)

mse_train=mean_squared_error(y_train, y_pred_train)
mse_val=mean_squared_error(y_val, y_pred_val)

print("Root Mean Squared error on Train data is:",math.sqrt(mse_train))
print("Root Mean Squared error on Val data is:",math.sqrt(mse_val))

Root Mean Squared error on Train data is: 1.0510422638776984
Root Mean Squared error on Val data is: 0.9040693685976806


In [ ]:
# Results with the best model
pred2 = model2.predict(X_test_new)
mse_test=mean_squared_error(y_test, pred2)
print("Root Mean Squared error on Test data is:",math.sqrt(mse_test))

Root Mean Squared error on Test data is: 0.9289066026075286


In [ ]:
spearmanr(y_test, pred2)

SignificanceResult(statistic=0.9960643067468167, pvalue=0.0)

In [ ]:
eval_df = pd.DataFrame({"Actual": y_test.values, "Predicted": pred2})
spearmanr_val = spearmanr(eval_df["Actual"], eval_df["Predicted"])

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x=eval_df["Actual"],
        y=eval_df["Predicted"],
        name='Actual vs. Predicted',
        mode='markers'
    )
)

fig.add_annotation(
    x=0.95,
    y=0.10,
    xref='paper',
    yref='paper',
    text='Spearman Rank Correlation = {:.3f}'.format(spearmanr_val.correlation),
    showarrow=False
)

fig.update_layout(
    title='Actual vs. Predicted',
    height=700
)

fig.show()

In [ ]:
r2_train = r2_score(y_train, y_pred_train)
r2_val = r2_score(y_val, y_pred_val)
r2_test = r2_score(y_test, pred2)
print('r2 score for train is', r2_train)
print('r2 score for val is', r2_val)
print('r2 score for test is', r2_test)

r2 score for train is 0.9924098344859508
r2 score for val is 0.9945614914305768
r2 score for test is 0.9944983313930463


In [ ]:
print('Mean Absolute Error Train:', metrics.mean_absolute_error(y_train, y_pred_train))
print('Mean Absolute Error Val:', metrics.mean_absolute_error(y_val, y_pred_val))
print('Mean Absolute Error Test:', metrics.mean_absolute_error(y_test, pred2))

Mean Absolute Error Train: 0.1436972944217904
Mean Absolute Error Val: 0.13239399717638273
Mean Absolute Error Test: 0.12612748704394938


## Model 3

In [ ]:
# Split the data into train and validation set
train_df_new, rem_df = train_test_split(train_df, train_size=0.8,random_state=101)

val_df, test_df = train_test_split(rem_df, test_size=0.5,random_state=101)

In [ ]:
X_train = train_df.drop(columns=['tm'])
y_train = train_df['tm']

X_val = val_df.drop(columns=['tm'])
y_val = val_df['tm']

X_test = test_df.drop(columns=['tm'])
y_test = test_df['tm']


In [ ]:
#XGBoost Model
model = XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=200, tree_method="gpu_hist",random_state=101)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=101, ...)

In [ ]:
rfecv = RFECV(estimator= model, step = 5, cv = 5, scoring='neg_mean_squared_error')
rfecv = rfecv.fit(X_train, y_train)

print("The optimal number of features:", rfecv.n_features_)
print('Selected features: %s' % list(X_train.columns[rfecv.support_]))

best_features = list(X_train.columns[rfecv.support_])

The optimal number of features: 27
Selected features: ['pH', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'Aromaticity', 'Molecular Weight', 'Instability Index', 'Hydrophobicity', 'Isoelectric Point', 'Charge']


In [ ]:
X_train_new = X_train[best_features]
X_val_new = X_val[best_features]
X_test_new = X_test[best_features]

In [ ]:
model3 = XGBRegressor(learning_rate=0.1, max_depth=20, n_estimators=250, tree_method="gpu_hist",random_state=123)
model3.fit(X_train_new, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=20, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=250, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=123, ...)

In [ ]:
y_pred_train = model3.predict(X_train_new)
y_pred_val = model3.predict(X_val_new)

mse_train=mean_squared_error(y_train, y_pred_train)
mse_val=mean_squared_error(y_val, y_pred_val)

print("Root Mean Squared error on Train data is:",math.sqrt(mse_train))
print("Root Mean Squared error on Val data is:",math.sqrt(mse_val))

Root Mean Squared error on Train data is: 1.0510422638776984
Root Mean Squared error on Val data is: 0.9881466740536117


In [ ]:
# Results with the best model
pred3 = model3.predict(X_test_new)
mse_test=mean_squared_error(y_test, pred3)
print("Root Mean Squared error on Test data is:",math.sqrt(mse_test))

Root Mean Squared error on Test data is: 1.0395822158150478


In [ ]:
spearmanr(y_test, pred3)

SignificanceResult(statistic=0.9945303235427085, pvalue=0.0)

In [ ]:
eval_df = pd.DataFrame({"Actual": y_test.values, "Predicted": pred3})
spearmanr_val = spearmanr(eval_df["Actual"], eval_df["Predicted"])

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x=eval_df["Actual"],
        y=eval_df["Predicted"],
        name='Actual vs. Predicted',
        mode='markers'
    )
)

fig.add_annotation(
    x=0.95,
    y=0.10,
    xref='paper',
    yref='paper',
    text='Spearman Rank Correlation = {:.3f}'.format(spearmanr_val.correlation),
    showarrow=False
)

fig.update_layout(
    title='Actual vs. Predicted',
    height=700
)

fig.show()

In [ ]:
r2_train = r2_score(y_train, y_pred_train)
r2_val = r2_score(y_val, y_pred_val)
r2_test = r2_score(y_test, pred3)
print('r2 score for train is', r2_train)
print('r2 score for val is', r2_val)
print('r2 score for test is', r2_test)

r2 score for train is 0.9924098344859508
r2 score for val is 0.9932823690855941
r2 score for test is 0.9928121864469538


In [ ]:
print('Mean Absolute Error Train:', metrics.mean_absolute_error(y_train, y_pred_train))
print('Mean Absolute Error Val:', metrics.mean_absolute_error(y_val, y_pred_val))
print('Mean Absolute Error Test:', metrics.mean_absolute_error(y_test, pred3))

Mean Absolute Error Train: 0.1436972944217904
Mean Absolute Error Val: 0.13897981267007062
Mean Absolute Error Test: 0.14828002580351357


## Model 4

In [ ]:
# Split the data into train and validation set
train_df_new, rem_df = train_test_split(train_df, train_size=0.8,random_state=99)

val_df, test_df = train_test_split(rem_df, test_size=0.5,random_state=101)

In [ ]:
X_train = train_df.drop(columns=['tm'])
y_train = train_df['tm']

X_val = val_df.drop(columns=['tm'])
y_val = val_df['tm']

X_test = test_df.drop(columns=['tm'])
y_test = test_df['tm']


In [ ]:
#XGBoost Model
model = XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=200, tree_method="gpu_hist",random_state=99)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=99, ...)

In [ ]:
rfecv = RFECV(estimator= model, step = 5, cv = 5, scoring='neg_mean_squared_error')
rfecv = rfecv.fit(X_train, y_train)

print("The optimal number of features:", rfecv.n_features_)
print('Selected features: %s' % list(X_train.columns[rfecv.support_]))

best_features = list(X_train.columns[rfecv.support_])

The optimal number of features: 27
Selected features: ['pH', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'Aromaticity', 'Molecular Weight', 'Instability Index', 'Hydrophobicity', 'Isoelectric Point', 'Charge']


In [ ]:
X_train_new = X_train[best_features]
X_val_new = X_val[best_features]
X_test_new = X_test[best_features]

In [ ]:
model4 = XGBRegressor(learning_rate=0.1, max_depth=20, n_estimators=250, tree_method="gpu_hist",random_state=123)
model4.fit(X_train_new, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=20, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=250, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=123, ...)

In [ ]:
y_pred_train = model4.predict(X_train_new)
y_pred_val = model4.predict(X_val_new)

mse_train=mean_squared_error(y_train, y_pred_train)
mse_val=mean_squared_error(y_val, y_pred_val)

print("Root Mean Squared error on Train data is:",math.sqrt(mse_train))
print("Root Mean Squared error on Val data is:",math.sqrt(mse_val))

Root Mean Squared error on Train data is: 1.0510422638776984
Root Mean Squared error on Val data is: 0.7799844450407125


In [ ]:
# Results with the best model
pred4 = model4.predict(X_test_new)
mse_test=mean_squared_error(y_test, pred4)
print("Root Mean Squared error on Test data is:",math.sqrt(mse_test))

Root Mean Squared error on Test data is: 1.104675751969522


In [ ]:
spearmanr(y_test, pred4)

SignificanceResult(statistic=0.9949242080998096, pvalue=0.0)

In [ ]:
eval_df = pd.DataFrame({"Actual": y_test.values, "Predicted": pred4})
spearmanr_val = spearmanr(eval_df["Actual"], eval_df["Predicted"])

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x=eval_df["Actual"],
        y=eval_df["Predicted"],
        name='Actual vs. Predicted',
        mode='markers'
    )
)

fig.add_annotation(
    x=0.95,
    y=0.10,
    xref='paper',
    yref='paper',
    text='Spearman Rank Correlation = {:.3f}'.format(spearmanr_val.correlation),
    showarrow=False
)

fig.update_layout(
    title='Actual vs. Predicted',
    height=700
)

fig.show()

In [ ]:
r2_train = r2_score(y_train, y_pred_train)
r2_val = r2_score(y_val, y_pred_val)
r2_test = r2_score(y_test, pred4)
print('r2 score for train is', r2_train)
print('r2 score for val is', r2_val)
print('r2 score for test is', r2_test)

r2 score for train is 0.9924098344859508
r2 score for val is 0.9958193965262252
r2 score for test is 0.9913499512205307


In [ ]:
print('Mean Absolute Error Train:', metrics.mean_absolute_error(y_train, y_pred_train))
print('Mean Absolute Error Val:', metrics.mean_absolute_error(y_val, y_pred_val))
print('Mean Absolute Error Test:', metrics.mean_absolute_error(y_test, pred4))

Mean Absolute Error Train: 0.1436972944217904
Mean Absolute Error Val: 0.09977688861034915
Mean Absolute Error Test: 0.15296366763923264


## Model 5

In [ ]:
# Split the data into train and validation set
train_df_new, rem_df = train_test_split(train_df, train_size=0.8,random_state=95)

val_df, test_df = train_test_split(rem_df, test_size=0.5,random_state=99)

In [ ]:
X_train = train_df.drop(columns=['tm'])
y_train = train_df['tm']

X_val = val_df.drop(columns=['tm'])
y_val = val_df['tm']

X_test = test_df.drop(columns=['tm'])
y_test = test_df['tm']

In [ ]:
#XGBoost Model
model = XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=200, tree_method="gpu_hist",random_state=99)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=99, ...)

In [ ]:
rfecv = RFECV(estimator= model, step = 5, cv = 5, scoring='neg_mean_squared_error')
rfecv = rfecv.fit(X_train, y_train)

print("The optimal number of features:", rfecv.n_features_)
print('Selected features: %s' % list(X_train.columns[rfecv.support_]))

best_features = list(X_train.columns[rfecv.support_])

The optimal number of features: 27
Selected features: ['pH', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'Aromaticity', 'Molecular Weight', 'Instability Index', 'Hydrophobicity', 'Isoelectric Point', 'Charge']


In [ ]:
X_train_new = X_train[best_features]
X_val_new = X_val[best_features]
X_test_new = X_test[best_features]

In [ ]:
model5 = XGBRegressor(learning_rate=0.1, max_depth=20, n_estimators=250, tree_method="gpu_hist",random_state=123)
model5.fit(X_train_new, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=20, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=250, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=123, ...)

In [ ]:
y_pred_train = model5.predict(X_train)
y_pred_val = model5.predict(X_val)

mse_train=mean_squared_error(y_train, y_pred_train)
mse_val=mean_squared_error(y_val, y_pred_val)

print("Root Mean Squared error on Train data is:",math.sqrt(mse_train))
print("Root Mean Squared error on Val data is:",math.sqrt(mse_val))

Root Mean Squared error on Train data is: 1.0510422638776984
Root Mean Squared error on Val data is: 0.9950483712042965


In [ ]:
# Results with the best model
pred5 = model5.predict(X_test)
mse_test=mean_squared_error(y_test, pred5)
print("Root Mean Squared error on Test data is:",math.sqrt(mse_test))

Root Mean Squared error on Test data is: 0.97164757779033


In [ ]:
spearmanr(y_test, pred5)

SignificanceResult(statistic=0.9950704646217198, pvalue=0.0)

In [ ]:
eval_df = pd.DataFrame({"Actual": y_test.values, "Predicted": pred5})
spearmanr_val = spearmanr(eval_df["Actual"], eval_df["Predicted"])

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x=eval_df["Actual"],
        y=eval_df["Predicted"],
        name='Actual vs. Predicted',
        mode='markers'
    )
)

fig.add_annotation(
    x=0.95,
    y=0.10,
    xref='paper',
    yref='paper',
    text='Spearman Rank Correlation = {:.3f}'.format(spearmanr_val.correlation),
    showarrow=False
)

fig.update_layout(
    title='Actual vs. Predicted',
    height=700
)

fig.show()

In [ ]:
r2_train = r2_score(y_train, y_pred_train)
r2_val = r2_score(y_val, y_pred_val)
r2_test = r2_score(y_test, pred5)
print('r2 score for train is', r2_train)
print('r2 score for val is', r2_val)
print('r2 score for test is', r2_test)

r2 score for train is 0.9924098344859508
r2 score for val is 0.9933603606247883
r2 score for test is 0.9936587214796955


In [ ]:
print('Mean Absolute Error Train:', metrics.mean_absolute_error(y_train, y_pred_train))
print('Mean Absolute Error Val:', metrics.mean_absolute_error(y_val, y_pred_val))
print('Mean Absolute Error Test:', metrics.mean_absolute_error(y_test, pred5))

Mean Absolute Error Train: 0.1436972944217904
Mean Absolute Error Val: 0.12430796164892975
Mean Absolute Error Test: 0.13316221427184147


## Combine XGBoost models

In [ ]:
# xgb_preds = np.round((pred1 + pred2 + pred3 + pred4 + pred5) / 5)
# mse_test_ensemble=mean_squared_error(y_test, xgb_preds)
# print("Ensemble Root Mean Squared Error on Test data is:",math.sqrt(mse_test_ensemble))

## LightBGM

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

In [ ]:
params = {
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": 1024,
    "max_depth": 500,
    "learning_rate": 0.1,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.9,
    "bagging_freq": 1,
    "min_data_in_leaf": 10,
    "lambda_l1": 0.000001,
    "lambda_l2": 0.000001,
    "seed": 123
}
num_rounds=1000

In [ ]:
lgmodel = lgb.train(params, train_data, num_rounds, valid_sets=[train_data, val_data], verbose_eval=100, early_stopping_rounds=50)

/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:181: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.

/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:239: UserWarning:

'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.



[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008077 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6477
[LightGBM] [Info] Number of data points in the train set: 28606, number of used features: 27
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 51.354995
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 1.48281	valid_1's rmse: 1.45704
[200]	training's rmse: 1.16805	valid_1's rmse: 1.15109
[300]	training's rmse: 1.1043	valid_1's rmse: 1.09343
[400]	training's rmse: 1.07891	valid_1's rmse: 1.08431
[500]	training's rmse: 1.06572	valid_1's rmse: 1.06691
Early stopping, best iteration is:
[513]	training's rmse: 1.06452	valid_1's rmse: 1.05716


In [ ]:
y_pred_train = lgmodel.predict(X_train)
y_pred_val = lgmodel.predict(X_val)

mse_train=mean_squared_error(y_train, y_pred_train)
mse_val=mean_squared_error(y_val, y_pred_val)

print("Root Mean Squared error on Train data is:",math.sqrt(mse_train))
print("Root Mean Squared error on Val data is:",math.sqrt(mse_val))

Root Mean Squared error on Train data is: 1.0645236653040733
Root Mean Squared error on Val data is: 1.0571595026077882


In [ ]:
# Results with the best model
lgbmpred = lgmodel.predict(X_test)
mse_test=mean_squared_error(y_test, lgbmpred)
print("Root Mean Squared error on Test data is:",math.sqrt(mse_test))

Root Mean Squared error on Test data is: 0.8865122562553175


In [ ]:
from sklearn.metrics import r2_score
r2_train = r2_score(y_train, y_pred_train)
r2_val = r2_score(y_val, y_pred_val)
r2_test = r2_score(y_test, lgbmpred)
print('r2 score for train is', r2_train)
print('r2 score for val is', r2_val)
print('r2 score for test is', r2_test)

r2 score for train is 0.9922138722026053
r2 score for val is 0.9920832903907838
r2 score for test is 0.9945970497021803


In [ ]:
from sklearn import metrics
print('Mean Absolute Error Train:', metrics.mean_absolute_error(y_train, y_pred_train))
print('Mean Absolute Error Val:', metrics.mean_absolute_error(y_val, y_pred_val))
print('Mean Absolute Error Test:', metrics.mean_absolute_error(y_test, lgbmpred))

Mean Absolute Error Train: 0.18575501395041727
Mean Absolute Error Val: 0.18396341726641038
Mean Absolute Error Test: 0.15612099797272785


In [ ]:
xgb_weight = 0.5
lgbm_weight = 0.5

# Combine predictions using weighted average
ensemble_preds = np.round((xgb_weight * pred1 + lgbm_weight * lgbmpred) / (xgb_weight + lgbm_weight))

In [ ]:
mse_test_ensemble=mean_squared_error(y_test, ensemble_preds)
print("Ensemble Root Mean Squared Error on Test data is:",math.sqrt(mse_test_ensemble))

Ensemble Root Mean Squared Error on Test data is: 0.9139143432367475


In [ ]:
mse_test_ensemble=mean_squared_error(y_test, ensemble_preds)
print("Ensemble Root Mean Squared Error on Test data is:",math.sqrt(mse_test_ensemble))

## Random Forest

In [ ]:
model = RandomForestRegressor(n_estimators=1024, max_depth=256, min_samples_split=20, min_samples_leaf=1, 
                               max_features='sqrt', bootstrap=False, random_state=42)

In [ ]:
# Train the model on training data
model.fit(X_train, y_train);

In [ ]:
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

mse_train=mean_squared_error(y_train, y_pred_train)
mse_val=mean_squared_error(y_val, y_pred_val)

print("Root Mean Squared error on Train data is:",math.sqrt(mse_train))
print("Root Mean Squared error on Val data is:",math.sqrt(mse_val))

In [ ]:
# Results with the best model
y_pred_test = model.predict(X_test)
mse_test=mean_squared_error(y_test, y_pred_test)
print("Root Mean Squared error on Test data is:",math.sqrt(mse_test))

## Linear Regression

In [ ]:
model_lr= LinearRegression()  
model_lr.fit(X_train_new, y_train)  

LinearRegression()

In [ ]:
y_pred_train = model_lr.predict(X_train_new)
y_pred_val = model_lr.predict(X_val_new)

mse_train=mean_squared_error(y_train, y_pred_train)
mse_val=mean_squared_error(y_val, y_pred_val)

print("Root Mean Squared error on Train data is:",math.sqrt(mse_train))
print("Root Mean Squared error on Val data is:",math.sqrt(mse_val))

Root Mean Squared error on Train data is: 10.348112314464077
Root Mean Squared error on Val data is: 10.203220983770612


In [ ]:
y_pred_test_lr= model_lr.predict(X_test_new)  
mse_test=mean_squared_error(y_test, y_pred_test_lr)
print("Root Mean Squared error on Test data is:",math.sqrt(mse_test))

Root Mean Squared error on Test data is: 10.288884345628928


## Catboost

In [ ]:
train_dataset = cb.Pool(X_train, y_train) 
val_dataset = cb.Pool(X_val, y_val) 
test_dataset = cb.Pool(X_test, y_test)

In [ ]:
model = CatBoostRegressor(iterations=2000,
                           learning_rate=0.1,
                           depth=13,
                           l2_leaf_reg=8,
                           loss_function='RMSE',
                           random_seed=99)

In [ ]:
# Fit the model to the training data
model.fit(X_train, y_train,
          eval_set=(X_val, y_val),
          early_stopping_rounds=100,
          verbose=100)

0:	learn: 11.5467354	test: 11.3790540	best: 11.3790540 (0)	total: 568ms	remaining: 18m 55s
100:	learn: 5.6928329	test: 5.6303214	best: 5.6303214 (100)	total: 56.3s	remaining: 17m 39s
200:	learn: 4.6204637	test: 4.5889351	best: 4.5889351 (200)	total: 1m 50s	remaining: 16m 28s
300:	learn: 3.7440021	test: 3.7262652	best: 3.7262652 (300)	total: 2m 44s	remaining: 15m 27s
400:	learn: 3.1785131	test: 3.1608956	best: 3.1608956 (400)	total: 3m 38s	remaining: 14m 29s
500:	learn: 2.7436795	test: 2.7290505	best: 2.7290505 (500)	total: 4m 31s	remaining: 13m 33s
600:	learn: 2.4568250	test: 2.4306967	best: 2.4306967 (600)	total: 5m 25s	remaining: 12m 37s
700:	learn: 2.2295078	test: 2.1989811	best: 2.1989811 (700)	total: 6m 18s	remaining: 11m 42s
800:	learn: 2.0618738	test: 2.0285788	best: 2.0285788 (800)	total: 7m 12s	remaining: 10m 47s
900:	learn: 1.9144854	test: 1.8775135	best: 1.8775135 (900)	total: 8m 6s	remaining: 9m 52s
1000:	learn: 1.8095591	test: 1.7691956	best: 1.7691956 (1000)	total: 9m	rem

In [ ]:
catpred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test,catpred)))
mae = (np.sqrt(mean_absolute_error(y_test, catpred)))
r2 = r2_score(y_test, catpred)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))
print('MAE: {:.2f}'.format(mae))

Testing performance
RMSE: 1.23
R2: 0.99
MAE: 0.75


In [ ]:
# catboost_model = CatBoostRegressor()

# # Define the hyperparameter grid for GridSearchCV
# param_grid = {
#     'iterations': [500, 1000],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'depth': [4, 6, 8],
#     'l2_leaf_reg': [1, 3, 5]
# }

# # Perform GridSearchCV
# grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, cv=5, verbose=0)
# grid_search.fit(X_train_check, y_train_check)

# # Print the best parameters and RMSE score
# print("Best parameters:", grid_search.best_params_)
# y_pred = grid_search.predict(X_test_check)
# rmse = np.sqrt(mean_squared_error(y_test_check, y_pred))
# print("RMSE:", rmse)